# Análisis de Conjuntos de trayectorias (Cluster)

1. Creación del archivo con las trayectorias a identificar conjuntos
2. Creación del archivo de control de cluster CCONTROL
1. Generación de conjuntos ejecutando `cluster` 
1. Visualizacón de la Variancia Espacial Total (TSV)
1. Determinación del número de conjuntos

In [1]:
%%bash -l
# Se crea el archivo INFILE
# se define el directorio de trabajo
DIR=enero
# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d $HOME/data/$DIR/conjuntos ];then 
    mkdir $HOME/data/$DIR/conjuntos ;fi

# se ubica en el directorio
cd $HOME/data/$DIR/conjuntos

#se crea el archivo INFILE seleccionando las trayectorias

ls  $HOME/data/$DIR/tdump1901*  > INFILE


Creación de `CCONTROL` y Ejecución del código para generación de cluster

In [2]:
%%bash -l

# se define el directorio de trabajo
DIR=enero
# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d $HOME/data/$DIR/conjuntos ];then 
    mkdir $HOME/data/$DIR/conjuntos  ;fi

# se ubica en el directorio DIR
cd $HOME/data/$DIR

# Directorio del hysplit
PGM="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"

#
################
# cluster1.scr
#################
  dur=12          # hours par realización clustering
  int=1           # Endpoint interval to use (hours)
  skip=1          # Skip trajectory interval
  ODIR="$HOME/data/$DIR/conjuntos/" # Directorio de saldia de Conjutos (con trailing /)
  cd $ODIR
  iproj=0
   
  echo $dur   > CCONTROL
  echo $int  >> CCONTROL 
  echo $skip >> CCONTROL 
  echo $ODIR >> CCONTROL 
  echo $iproj >> CCONTROL

# Realizando la conjunción 
  rm -f TCLUS TNOCLUS CLUSTER DELPCT CLUSTERno CMESSAGE
  $PGM/exec/cluster 
  if [ ! -f CLUSTER -o ! -f DELPCT ]; then
     echo "ERROR - cluster failed"
     exit
  fi
  rm TNOCLUS   # not used
  rm TCLUS     # not used


 Started Cluster (Version: April 2015)
 Number of trajectories in INFILE:         124
Pass    1 out of  112
Pass    2 out of  112
Pass    3 out of  112
Pass    4 out of  112
Pass    5 out of  112
Pass    6 out of  112
Pass    7 out of  112
Pass    8 out of  112
Pass    9 out of  112
Pass   10 out of  112
Pass   11 out of  112
Pass   12 out of  112
Pass   13 out of  112
Pass   14 out of  112
Pass   15 out of  112
Pass   16 out of  112
Pass   17 out of  112
Pass   18 out of  112
Pass   19 out of  112
Pass   20 out of  112
Pass   21 out of  112
Pass   22 out of  112
Pass   23 out of  112
Pass   24 out of  112
Pass   25 out of  112
Pass   26 out of  112
Pass   27 out of  112
Pass   28 out of  112
Pass   29 out of  112
Pass   30 out of  112
Pass   31 out of  112
Pass   32 out of  112
Pass   33 out of  112
Pass   34 out of  112
Pass   35 out of  112
Pass   36 out of  112
Pass   37 out of  112
Pass   38 out of  112
Pass   39 out of  112
Pass   40 out of  112
Pass   41 out of  112
Pass   42 ou

## Identificación del numero de conjuntos

In [4]:
%%bash -l
# se define  directorio de trabajo
DIR=enero
# se define directorio del hysplit
PGM="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"

cd $HOME/data/$DIR/conjuntos/

# Titulo del grafico

  ilab1=NUCLEO 

# TSV grafico de la Variancia Espacial Total (TSV)
  $PGM/exec/clusplot +g0 -iDELPCT -oclusplot.ps -l${ilab1}
#
# Creacion de figura para visualizar
  if [ ! -s clusplot.ps ]; then
     echo "ERROR - clusplot fallo"
     exit
  fi
convert clusplot.ps clusplot.jpg


 Started Cluster TSV Drawing (Version: November 2009)
 Complete Clusplot


De la figura se selecciona el número donde se da el quiebre de la pendiente mas pronunciada a la de menor 

Determinación del número final de conjuntos

In [5]:
%%bash -l

# Se define  directorio de trabajo
DIR=enero
# se define directorio del hysplit
PGM="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"

cd $HOME/data/$DIR/conjuntos


# variables for clusend (determining final number of clusters) --
  mint=15    # min number traj 
  minc=3     # min number of clusters
  maxc=10    # max number of clusters
  pct=30     # min % change in DELPCT from 1 step to the next
             #     ie breakpoint to indicate end of clustering
    
   nc=5      # numero de conjuntos

# determine final number of clusters
  rm -f CLUSEND
  if $PGM/exec/clusend -iDELPCT -n${minc} -a${maxc} -t${mint} -p${pct} -oCLUSEND; then
     echo $PGM/exec/clusend OK 1st try
  else
     # relax criteria to help get # clusters
     pct=`expr $pct - 10`
     $PGM/exec/clusend -iDELPCT -n${minc} -a${maxc} -t${mint} -p${pct} -oCLUSEND
  fi
  if [ ! -s CLUSEND ]; then
     echo "ERROR - clusend failed"
     exit
  fi

# generate list of trajs in each cluster
  rm -f CLUSLIST_${nc}
  $PGM/exec/cluslist -iCLUSTER -n${nc} -oCLUSLIST_${nc}
  if [ ! -s CLUSLIST_${nc} ]; then
     echo "ERROR - cluslist failed"
     exit
  fi
# add trajs not clustered (cluster #0) to CLUSLIST
  if [ -s CLUSTERno ]; then       
     cat CLUSLIST_${nc} >> CLUSTERno
     mv CLUSTERno CLUSLIST_${nc}
  fi

# create TRAJ.INP.Cxx file for each cluster (list of trajs) 
  rm -f TRAJ.INP*
  if $PGM/exec/clusmem -iCLUSLIST_${nc}; then

     ##############################################
     # for each cluster 
     ##############################################
     run=-1					# for cluster #0 
     while [ $run -lt $nc ]; do
        run=`expr $run + 1`
        if [ -f TRAJ.INP.C${run}_${nc} ]; then	# cluster #0 may not exist
           ##############################################
           # create tdump file of all trajs
           ##############################################
           $PGM/exec/merglist -i+TRAJ.INP.C${run}_${nc} -omdump -ptdump
           if [ -s mdump.tdump ]; then
               mv mdump.tdump C$run.tdump
           else
               echo "ERROR - merglist TRAJ.INP.C${run}_${nc}"
               exit 
           fi
           ##############################################
           # compute mean 
           ##############################################
           $PGM/exec/trajmean -i+TRAJ.INP.C${run}_${nc} 
           if [ -s tmean ]; then
              mv tmean C${run}mean.tdump
           else
              echo "ERROR - trajmean TRAJ.INP.C${run}_${nc}"
              exit 
           fi
        fi
     done

     ##############################################
     # make one tdump for all means
     ##############################################
     # for each cluster  (not zero)
     run=0
     rm -f MEAN.LIST
     while [ $run -lt $nc ]; do
        run=`expr $run + 1`
        echo "C${run}mean.tdump" >> MEAN.LIST
     done

     $PGM/exec/merglist -i+MEAN.LIST -omdump -ptdump
     if [ -s mdump.tdump ]; then
        mv mdump.tdump Cmean.tdump
     else
        echo "ERROR - merglist MEAN.LIST"
        exit 
     fi
     rm -f MEAN.LIST

  else
     echo "ERROR - clusmem"
     exit 
  fi


 Started Clusend (Version: November 2009)
 Complete Clusend
/home/agustin/data/hysplit.v5.2.0_CentOS7.9.2009/exec/clusend OK 1st try
 Started Cluslist (Version: November 2007)
 Complete Cluslist
 Started Clusmem (Version: November 2007)
 Complete Clusmem
 Started Trajectory Endpts Merglist (Version: Nov 2007)
 Trajectory one kpts:           9
 Input file, kpts:           2          10
 Input file, kpts:           3          10
 Input file, kpts:           4          10
 Input file, kpts:           5          10
 Input file, kpts:           6          10
 Input file, kpts:           7          12
 Input file, kpts:           8          12
 Input file, kpts:           9          12
 Input file, kpts:          10          12
 Input file, kpts:          11          12
 Processing: /home/agustin/data/enero/tdump19010116  
 Processing: /home/agustin/data/enero/tdump19010120  
 Processing: /home/agustin/data/enero/tdump19010202  
 Processing: /home/agustin/data/enero/tdump19010205  
 Processi

## Visualización de las trayectorias


In [7]:
%%bash -l
# Se define  directorio de trabajo
DIR=enero
# se define directorio del hysplit
PGM="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"

nc=5

cd $HOME/data/$DIR/conjuntos
echo "'$PGM/graphics/mexico.shp' 0 0.01 0.4 0.6 0.8" >shapefiles_mex.txt

# plot trajectories for each cluster
  run=-1                 # for cluster #0 
  while [ $run -lt $nc ]; do
     run=`expr $run + 1`
     if [ -s C$run.tdump ]; then 
        rm -f LABELS.CFG
        echo "'TITLE&','TRAJECTORIES IN CLUSTER ${run} of ${nc}&'"  >LABELS.CFG
        $PGM/exec/trajplot +g0 -iC$run.tdump -jshapefiles_mex.txt -e$dur -z80 -k0 -l0 -v4
        if [ -s trajplot.ps ]; then
           convert trajplot.ps trajplot_C$run.jpg
           echo "Enter to continue ...";#  read x
        else
           echo "Error in trajplot +g0 -iC$run.tdump" 
           exit 
        fi
     fi
  done
  rm -f LABELS.CFG
 #
 # compute/plot mean traj for each cluster
#
tdump=Cmean.tdump
echo ${tdump} tedump
if [ -s ${tdump} ]; then
     rm -f LABELS.CFG
     echo "'TITLE&','CLUSTER MEAN TRAJECTORIES&'" >LABELS.CFG
     $PGM/exec/trajplot +g0 -jshapefiles_mex.txt -iCmean.tdump -e$dur -z80 -k${nc}:12345 -l-12 -v4
        convert trajplot.ps trajmean.jpg
    
else
        echo "ERROR - trajplot +g0 -iCmean.tdump"
        exit 
fi
     rm -f LABELS.CFG

# ---------------------------------------------------------------------
# cleanup
# rm -f CCONTROL
# rm -f CLUSTER CLUSTERno CMESSAGE DELPCT
# rm -f CLUSLIST_* CLUSEND 
# rm -f TRAJ.INP.C?_?
# rm -f fdump8309????
# rm -f  *.tdump


 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Complete Trajplot
Enter to continue ...
Cmean.tdump tedump
 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Reading CLUSLIST_5                                                                      
 